In [1]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from imblearn.over_sampling import SMOTE

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score




MNCAATourneySeeds = pd.read_csv('/kaggle/input/basketball-prediction/MNCAATourneySeeds.csv')
MRegularSeason = pd.read_csv('/kaggle/input/basketball-prediction/MRegularSeasonCompactResults.csv')
MTourney = pd.read_csv('/kaggle/input/basketball-prediction/MNCAATourneyCompactResults.csv')
MTourneyDetail = pd.read_csv('/kaggle/input/basketball-prediction/MNCAATourneyDetailedResults.csv')
seeds_2024 = pd.read_csv("/kaggle/input/basketball-prediction/2024_tourney_seeds.csv")
sub = pd.read_csv("/kaggle/input/basketball-prediction/sample_submission.csv")
slot = pd.read_csv("/kaggle/input/basketball-prediction/MNCAATourneySlots.csv")

2024-04-09 16:03:15.324066: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 16:03:15.324246: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 16:03:15.528545: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# MTourney
seed_mapping = dict(zip(zip(MNCAATourneySeeds['Season'], MNCAATourneySeeds['TeamID']), MNCAATourneySeeds['Seed']))
MTourneyDetail['WSeed'] = MTourneyDetail.apply(lambda row: seed_mapping.get((row['Season'], row['WTeamID']), pd.NA), axis=1)
seed_mapping = dict(zip(zip(MNCAATourneySeeds['Season'], MNCAATourneySeeds['TeamID']), MNCAATourneySeeds['Seed']))
MTourneyDetail['LSeed'] = MTourneyDetail.apply(lambda row: seed_mapping.get((row['Season'], row['LTeamID']), pd.NA), axis=1)

MTourneyDetail['WSeed_Num'] = MTourneyDetail['WSeed'].str.extract(r'(\d+)').astype(int)
MTourneyDetail['LSeed_Num'] = MTourneyDetail['LSeed'].str.extract(r'(\d+)').astype(int)
MTourneyDetail

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LDR,LAst,LTO,LStl,LBlk,LPF,WSeed,LSeed,WSeed_Num,LSeed_Num
0,2003,134,1421,92,1411,84,N,1,32,69,...,28,16,15,5,0,22,X16b,X16a,16,16
1,2003,136,1112,80,1436,51,N,0,31,66,...,26,12,17,10,3,15,Z01,Z16,1,16
2,2003,136,1113,84,1272,71,N,0,31,59,...,22,11,12,2,5,18,Z10,Z07,10,7
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,20,21,6,6,21,Z11,Z06,11,6
4,2003,136,1143,76,1301,74,N,1,27,64,...,26,16,14,5,8,19,W08,W09,8,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,2023,146,1274,88,1400,81,N,0,29,49,...,14,20,13,6,1,23,Y05,Y02,5,2
1311,2023,146,1361,57,1166,56,N,0,25,66,...,24,9,9,3,2,11,X05,X06,5,6
1312,2023,152,1163,72,1274,59,N,0,28,57,...,17,10,9,8,1,12,Z04,Y05,4,5
1313,2023,152,1361,72,1194,71,N,0,25,57,...,24,6,9,6,2,17,X05,W09,5,9


In [3]:
def get_team_averages(_games):
    _stats_keys = np.array(["FGA", "FGM", "FGA3", "FGM3", "FTA", "FTM", "OR", "DR", "TO"])
    base_stats_w = np.array(["Season", "WTeamID", "LTeamID", "WScore", "LScore"])
    base_stats_l = np.array(["Season", "LTeamID", "WTeamID", "LScore", "WScore"])
    
    win_keys = np.concatenate((base_stats_w, np.char.add("W", _stats_keys)))
    lose_keys = np.concatenate((base_stats_l, np.char.add("L", _stats_keys)))

    wins = _games[win_keys].assign(GameResult="W", Won = 1).rename(columns={"WTeamID":"TeamID", "WScore":"Score", "LTeamID":"OppTeamID", "LScore":"OppScore"}).rename(columns={win_keys[i+5]:_stats_keys[i] for i in range(len(_stats_keys))})
    losses = _games[lose_keys].assign(GameResult="L", Won = 0).rename(columns={"LTeamID":"TeamID", "LScore":"Score", "WTeamID":"OppTeamID", "WScore":"OppScore"}).rename(columns={lose_keys[i+5]:_stats_keys[i] for i in range(len(_stats_keys))})

    games_by_team = pd.concat([wins, losses], axis=0)

    games_by_team["ScoreDiff"] = (games_by_team["Score"]-games_by_team["OppScore"])
    games_by_team["FGAcc"] = (games_by_team["FGM"]/games_by_team["FGA"])
    games_by_team["FG3Acc"] = (games_by_team["FGM3"]/games_by_team["FGA3"])
    games_by_team["FTAcc"] = (games_by_team["FTM"]/games_by_team["FTA"])
    games_by_team["WinPrc"] = (games_by_team["Won"])

    season_stats_keys = np.array(["Score", "ScoreDiff", "FGAcc", "FG3Acc", "FTAcc", "WinPrc"])

    team_averages = games_by_team.groupby(["Season", "TeamID"])[season_stats_keys].mean().reset_index()
    team_averages = team_averages[np.concatenate((["Season", "TeamID"], season_stats_keys), axis=0)].rename(columns={team_averages.columns[i+2]:np.char.add("Avg", season_stats_keys)[i] for i in range(len(season_stats_keys))})
    
    
    return team_averages


In [4]:
x = get_team_averages(MTourneyDetail)


In [5]:

def treat_seed(seed):
    return int(re.sub("[^0-9]", "", seed))

def map_days(day):
    if day == 134 or map == 135:
        return 0
    elif day == 136 or map == 137:
        return 1
    elif day == 138 or map == 139:
        return 2
    elif day == 143 or map == 144:
        return 3
    elif day == 145 or map == 146:
        return 4
    elif day == 152:
        return 5
    else:
        return 6
    
def map_rounds(_round):
    if _round==0:
        return 135
    elif _round==1:
        return 137
    elif _round==2:
        return 139
    elif _round==3:
        return 144
    elif _round==4:
        return 146
    elif _round==5:
        return 152
    else:
        return 154

In [6]:

def get_tourney_matchups(_tourney, _seeds):
    tournament = _tourney[["Season", "WTeamID", "LTeamID", "DayNum"]]
    tournament = pd.merge(tournament, _seeds, left_on=["Season", "WTeamID"], right_on=["Season", "TeamID"]).rename(columns={"Seed":"WSeed"}).drop(["TeamID"], axis=1)
    tournament = pd.merge(tournament, _seeds, left_on=["Season", "LTeamID"], right_on=["Season", "TeamID"]).rename(columns={"Seed":"LSeed"}).drop(["TeamID"], axis=1)

    tournament["WSeed"] = tournament["WSeed"].apply(treat_seed)
    tournament["LSeed"] = tournament["LSeed"].apply(treat_seed)
    daynums = tournament["DayNum"]
    tournament["Round"] = tournament["DayNum"].apply(map_days)
    tournament = tournament.drop(["DayNum"], axis=1)
    tournament["DayNum"] = daynums

    tournament["Upset"] = (tournament["WSeed"] > tournament["LSeed"]).astype(int)
    tournament["SeedDiff"] = abs(tournament["WSeed"] - tournament["LSeed"])

    straight = tournament[(tournament["Upset"]==False) & (tournament["SeedDiff"] != 0)].rename(columns={"WTeamID":"ATeam", "LTeamID":"BTeam", "WSeed":"ASeed", "LSeed":"BSeed"})
    upset = tournament[tournament["Upset"]==True].rename(columns={"LTeamID":"ATeam", "WTeamID":"BTeam", "LSeed":"ASeed", "WSeed":"BSeed"})
    even = tournament[tournament["SeedDiff"] == 0]

    matchups = pd.concat([straight, upset])
    
    return matchups

In [7]:
def get_merged_matchups(_matchups, _team_averages):
    team_averages_A = _team_averages.add_suffix("_A")
    team_averages_B = _team_averages.add_suffix("_B")

    _merged_matchups = _matchups.merge(team_averages_A, left_on=["Season", "ATeam"], right_on=["Season_A", "TeamID_A"])
    _merged_matchups = _merged_matchups.drop(["TeamID_A", "Season_A"], axis=1)

    _merged_matchups = _merged_matchups.merge(team_averages_B, left_on=["Season", "BTeam"], right_on=["Season_B", "TeamID_B"])
    _merged_matchups = _merged_matchups.drop(["TeamID_B", "Season_B"], axis=1)
    
    return _merged_matchups

In [8]:
def get_data(_merged_matchups, _test_valid_size=.1, _shuffled=True):
    _train_data = _merged_matchups.drop(["Season", "ATeam", "BTeam"], axis=1)
    if(_shuffled):
        _train_data = _train_data.sample(frac = 1)

    _X = _train_data.drop(["Upset"], axis=1).reset_index(drop=True)
    _y = _train_data[["Upset"]].reset_index(drop=True)

    oversample = SMOTE()
    _X, _y = oversample.fit_resample(_X, _y)

    _scaler = MinMaxScaler(feature_range=(-1,1))
    _X = _scaler.fit_transform(_X)

    _X_train, _X_testvalid, _y_train, _y_testvalid = train_test_split(_X, _y, test_size=_test_valid_size, random_state=8) 
    _X_valid, _X_test, _y_valid, _y_test = train_test_split(_X_testvalid, _y_testvalid, test_size=.5, random_state=8)
    _y_train, _y_valid, _y_test = np.array(_y_train).ravel(), np.array(_y_valid).ravel(), np.array(_y_test).ravel()
    
    return _X, _y, _X_train, _X_valid, _X_test, _y_train, _y_valid, _y_test, _scaler

In [9]:
match = get_tourney_matchups(MTourneyDetail, MNCAATourneySeeds)


In [10]:
merged_matchups_m = get_merged_matchups(match, x)

r1_upset = merged_matchups_m[(merged_matchups_m["Round"]== 1) & (merged_matchups_m["SeedDiff"] >= 5)]
later_upset = merged_matchups_m[(merged_matchups_m["Round"]!= 1) & (merged_matchups_m["SeedDiff"] >= 5)]

initial_close = merged_matchups_m[(merged_matchups_m["Round"]< 4) & (merged_matchups_m["SeedDiff"] < 5)]
later_close = merged_matchups_m[(merged_matchups_m["Round"]>=4) & (merged_matchups_m["SeedDiff"] < 5)]
r2 = merged_matchups_m[(merged_matchups_m["Round"]==2)]
X_m, y_m, X_train_m, X_valid_m, X_test_m, y_train_m, y_valid_m, y_test_m, scaler_m = get_data(merged_matchups_m)

In [11]:
model_m = Sequential([
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")
])
model_m.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])


In [12]:
history_m = model_m.fit(X_train_m, y_train_m, epochs=10, batch_size=1)

Epoch 1/10
1589/1589 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8034 - loss: 0.4366
Epoch 2/10
1589/1589 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9885 - loss: 0.0645
Epoch 3/10
1589/1589 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9966 - loss: 0.0346
Epoch 4/10
1589/1589 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9968 - loss: 0.0239
Epoch 5/10
1589/1589 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9987 - loss: 0.0174
Epoch 6/10
1589/1589 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9954 - loss: 0.0142
Epoch 7/10
1589/1589 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9996 - loss: 0.0118
Epoch 8/10
1589/1589 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9972 - loss: 0.0120
Epoch 9/10
1589/1589 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9972 - loss: 0.0128
Epoch 10/10
1589/1589 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9991 - loss: 0.0071


In [13]:

# Make predictions
predictions = model_m.predict(X_test_m)
predictions

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step 


array([[9.9513078e-01],
       [9.9998814e-01],
       [2.1795744e-10],
       [2.1397202e-06],
       [9.9995136e-01],
       [1.0000000e+00],
       [9.9995536e-01],
       [3.6030608e-12],
       [9.9999994e-01],
       [8.8640956e-11],
       [9.9994802e-01],
       [9.9998063e-01],
       [9.9999750e-01],
       [7.5515476e-05],
       [2.3972190e-08],
       [7.9425455e-10],
       [9.9989241e-01],
       [9.9864262e-01],
       [9.9999946e-01],
       [1.4589469e-10],
       [9.9967217e-01],
       [9.9854082e-01],
       [3.8015901e-06],
       [2.5661320e-08],
       [1.6268963e-10],
       [2.6456365e-02],
       [1.4623299e-06],
       [9.0711126e-08],
       [2.1807358e-09],
       [3.1602764e-07],
       [9.9988014e-01],
       [9.6166366e-01],
       [9.9999988e-01],
       [9.9999988e-01],
       [9.9999893e-01],
       [9.9999899e-01],
       [4.7536378e-06],
       [1.1776394e-10],
       [3.5762940e-05],
       [2.0954189e-08],
       [9.3420368e-01],
       [5.425504

In [14]:
nnpreds = model_m.predict(X_test_m)
nnpreds = (nnpreds>.8).astype(int)

print('precision: %.2f' % precision_score(y_test_m, nnpreds))
print('recall: %.2f' % recall_score(y_test_m, nnpreds))
print('f1_score: %.2f' % f1_score(y_test_m, nnpreds))
pd.DataFrame(data=nnpreds)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
precision: 1.00
recall: 0.96
f1_score: 0.98


,0
0,1
1,1
2,0
3,0
4,1
...,...
84,0
85,1
86,1
87,0
